In [ ]:
from ctgan import CTGAN
from ctgan import load_demo
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch
import tensorflow as tf
print(torch.cuda.is_available())
import pandas as pd
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split


In [ ]:

import json
import pandas as pd
import matplotlib.pyplot as plt

# Load log file
def load_log(log_file):
    with open(log_file, 'r') as f:
        log_data = [json.loads(line.strip()) for line in f]
    return pd.DataFrame(log_data)

# Add annotations for bars and lines
def add_annotations(ax, x, y, is_line=False):
    for i, value in enumerate(y):
        if is_line:
            ax.annotate(f"{value:.2f}", xy=(x[i], y[i]), ha='center', va='bottom', fontsize=9, color='red')
        else:
            ax.annotate(f"{value:.2f}", xy=(x[i], y[i]), ha='center', va='bottom', fontsize=9, color='blue')

# Plot combined metrics
def plot_metrics_comparison(df, save_path):
    plt.figure(figsize=(14, 8))

    x_labels = [f"{row['model']} ({row['test_dataset']})" for _, row in df.iterrows()]
    x_positions = range(len(x_labels))
    
    fig, ax1 = plt.subplots(figsize=(14, 8))

    # Bar: Training Time
    bar_width = 0.4
    bars = ax1.bar(x_positions, df['train_time'], width=bar_width, alpha=0.6, label="Training Time (s)", color='blue')
    ax1.set_ylabel("Training Time (s)", color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.set_xticks(x_positions)
    ax1.set_xticklabels(x_labels, rotation=45, ha='right')

    # Add annotations to bars
    add_annotations(ax1, x_positions, df['train_time'])

    # Line: Accuracy
    ax2 = ax1.twinx()
    ax2.plot(x_positions, df['Accuracy'], marker='o', color='red', label="Accuracy")
    ax2.set_ylabel("Accuracy", color='red')
    ax2.tick_params(axis='y', labelcolor='red')

    # Add annotations to line
    add_annotations(ax2, x_positions, df['Accuracy'], is_line=True)

    # Title and Legend
    plt.title("Training Time and Accuracy Comparison Across Models")
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
    plt.tight_layout()
    plt.savefig(save_path)
    print(f"Combined metrics comparison chart saved at {save_path}")
    plt.show()

# Additional detailed analysis
def analyze_metrics(df):
    print("\nDetailed Metric Analysis:")
    for metric in ['Accuracy', 'F1 Score', 'AUC']:
        best_model = df.loc[df[metric].idxmax()]
        print(f"Best model for {metric}: {best_model['model']} ({best_model['train_dataset']} -> {best_model['test_dataset']}) "
              f"with {metric}: {best_model[metric]:.2f}")


In [ ]:
log_file = 'experiment_log_with_time.json'  # Path to the log file
save_path = 'combined_metrics_comparison.png'  # Path to save the chart
df = load_log(log_file)

for col in ['train_time', 'Accuracy', 'F1 Score', 'AUC']:
    df[col] = df[col].astype(float)
    plot_metrics_comparison(df, save_path)
    analyze_metrics(df)
    